# Day100
## 訓練卷積神經網路的細節與技巧 - 轉移學習 (Transfer learning)
- 資料量不足時，遷移學習也是很常見的方法
- 神經網路訓練前的初始參數是隨機產生的，不具備任何意義
- 透過在其他龐大資料集上訓練好的模型參數，我們使用這個參數當成起始點，改⽤在⾃己的資料集上訓練！

### 為何可以用遷移學習？
- CNN 淺層的過濾器 (filter) 是用來偵測線條與顏色等簡單的元素。因此不管圖像是什麼類型，基本的組成應該要是一樣的。
- 大型資料集訓練好的參數具有完整的顏色、線條 filters，從此參數開始，我們訓練在⾃⼰的資料集中，逐步把 filters 修正為適合自己資料集的結果。

### Transfer learning in Keras: ResNet-50
```python
from keras.applications.resnet50 import ResNet50
restnet_model = ResNet50(input_shape=(224, 224, 3),
                         weights="imagenet",
                         pool="avg",
                         include_top=False)
```
> 我們使用了 ResNet50 網路結構，其中可以看到
1. ```weights = "imagenet"``` : 代表我們使用從 imagenet 訓練好的參數來初始化，並指定輸入的影像大小為 (224,224,3)。
2. ```pooling = "avg"``` : 代表最後一層使用[Global Average pooling](https://blog.csdn.net/Losteng/article/details/51520555)，把 featuremaps 變成一維的向量。
3. ```include_top = False``` : 代表將原本的 Dense layer 拔掉，因為原本這個網路是⽤來做 1000 個分類的模型，我們必須替換成⾃己的 Dense layer 來符合我們自己資料集的類別數量。

```python
last_featuremaps = resnet_model.output
flatten_featuremaps = Flatten()(last_featuremaps)
output = Denst(num_class)(flatten_featuremaps)

New_resnet_model = Model(inputs=[resnet_model.input], outputs=[output])
```

> 第一區塊的模型我們已經設定成沒有 Dense layers，且最後一層做 GAP（Global Average pooling）
1. 使用resnet_model.output 我們就可以取出最後一層的 featuremap
2. 將其使用 Flatten 攤平後
3. 再接上我們的 Dense layer，神經元數量與資料集的類別數量⼀致，重建立模型，就可以得到⼀個新的 ResNet-50 模型，且參數是根據 ImageNet ⼤型資料集預訓練好的


整體流程如下圖，我們保留 Trained convolutional base，並新建 New classifier (Dense 的部分)，最後 convolutional base 是否要 frozen (不訓練) 則是要看資料集與預訓練的 ImageNet 是否相似，如果差異很大則建議訓練時不要 frozen，讓 CNN 的參數可以繼續更新
<img src="https://camo.githubusercontent.com/2de162f6f3587a0422389355909594413ae17742/68747470733a2f2f73332e616d617a6f6e6177732e636f6d2f626f6f6b2e6b657261732e696f2f696d672f6368352f7377617070696e675f66635f636c61737369666965722e706e67" width='500' height='500'>

In [1]:
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
# 建立 ResNet 模型
model = resnet(input_shape=(32,32,3)) 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_____________________________________

In [4]:
batch_size = 64 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 30 # 訓練整個資料集共 30個循環

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 68s 1ms/step - loss: 1.8500 - acc: 0.5034 - val_loss: 1.6021 - val_acc: 0.5668
Epoch 2/30
50000/50000 [==============================] - 51s 1ms/step - loss: 1.3460 - acc: 0.6503 - val_loss: 1.5850 - val_acc: 0.5716
Epoch 3/30
50000/50000 [==============================] - 52s 1ms/step - loss: 1.1379 - acc: 0.7144 - val_loss: 1.2323 - val_acc: 0.6771
Epoch 4/30
50000/50000 [==============================] - 52s 1ms/step - loss: 1.0006 - acc: 0.7567 - val_loss: 1.4532 - val_acc: 0.6174
Epoch 5/30
50000/50000 [==============================] - 52s 1ms/step - loss: 0.9064 - acc: 0.7884 - val_loss: 1.0882 - val_acc: 0.7248
Epoch 6/30
50000/50000 [==============================] - 52s 1ms/step - loss: 0.8345 - acc: 0.8113 - val_loss: 1.1337 - val_acc: 0.7025
Epoch 7/30
50000/50000 [==============================] - 52s 1ms/step - loss: 0

## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠訓練!

最後一天的作業請閱讀這篇[非常詳盡的文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以 [參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

## 進階挑戰!
有志以成為機器學習工程師為目標的同學們可以參加這個挑戰，這[網站](https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html#43494641522d3130)記錄了歷年來 Cifar-10 中排名最高的論文，請試著閱讀論文並撰寫出相對應的程式碼，復現出論文的結果。

這樣的能力在機器學習領域中是非常重要的，具備閱讀他人論文並實現的能力，可為自己在履歷上增添不少分數，當然難度也相當高，若是不透徹了解文章內容或是程式能力不夠紮實，可是不能復現別人辛苦的研究成果的喔! 就請各位同學好好努力，往自己的機器學習之路邁進吧:)